# Contents <a id='top'></a>

1. <a href=#rv>Random Variables</a>
    1. <a href=#randomtext>Generating Random Text</a>
1. <a href=#des>Discrete Event Simulations</a>
    1. <a href=#des>Sandwich Shop</a>
    1. <a href=#simpy>About SimPy</a>
    1. <a href=#rc>Reneging Customers</a>
1. <a href=#ref>References and Links</a>

In [ ]:
import simpy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from numpy.random import default_rng
from scipy.stats import binom, bernoulli, norm, expon, uniform

import myscripts.simulation as ds

%matplotlib inline

<a id='rv'></a>
# 1. Random Variables
<a href=#top>(back to top)</a>

In statistics, we use random variables to describe the probabilistic behaviour of phenomenon. Random variables are real numbers that represent the phenomena we observe. For instance, we might let $X$ represent a coin toss, with $X=1$ representing Heads and $X=0$ representing Tails.

Random variables come with a rule (or function) that prescribes the probabilities with which it takes on particular values. For instance, if we had a fair coin, then the rule would be that 
$$
P(X=1) = P(X=0) = \frac{1}{2}
$$

On the other hand, a biased coin might follow the rule
$$
P(X=1) = \frac{2}{3},\quad P(X=0) = \frac{1}{3}
$$

This rule tells us which events are more likely, and which are less likely.

### Discrete vs. Continuous Random Variables.

Discrete random variables take on only a countable number of values. Examples are:
* A coin toss ($\{0, 1\}$)
* The number of taxis passing by a particular junction between 12noon and 1pm. ($\{0, 1, 2, \ldots\}$)
* The number of coin tosses until we observe Heads. ($\{1, 2, 3, \ldots\}$)

Discrete random variables are defined by their probability mass function (pmf), which is just a table or a function describing $P(X=i)$ for all possible $i$ values. Once we know the pmf of a random variable, we know everything about it - the mean, variance, quantiles, maximum values, etc.

We can visualise a pmf using a bar-chart. Here is the pmf for the above biased coin.

In [ ]:
plt.bar([0,1], [1/3, 2/3], tick_label=['0', '1'], width=0.3);
plt.xlim(-1,2); plt.ylim(0,1); plt.title('pmf of \'X\' random variable' );

Here is the pmf for a random variable representing the total number of Heads after 10 tosses of that same coin. Suppose we call that new random variable $Y$.

In [ ]:
probs = binom.pmf(np.arange(0, 11), n=10, p=2/3)
plt.bar(np.arange(0, 11), probs);plt.ylim(0,0.3); plt.title('pmf of no. of coin tosses');

If we wish to find the probability of events, for instance, $P(Y \le 4)$, we sum up the heights of the bars.

Continuous random variables are defined by what is known as a probability density function (pdf). Continuous random variables can take on an uncountable number of values, for instance, all real numbers in the interval $[0,1]$.

In [ ]:
x = np.linspace(-3, 3, num=100)
y = norm.pdf(x)
plt.figure(figsize=(12,4), facecolor='0.9')
plt.subplot(131)
plt.plot(x,y)
plt.ylim(0,1)
plt.title('Normal pdf');

y = expon.pdf(x[x>0])
plt.subplot(132)
plt.plot(x[x>0],y)
plt.ylim(0,1)
plt.title('Exponential pdf');

x = np.linspace(0, 1, num=100)
y = uniform.pdf(x)
plt.subplot(133)
plt.plot(x,y)
plt.ylim(0,1.1)
plt.title('Uniform pdf');

### Generating Random Variates

With a computer, we can generate random variables from almost any distribution. There are built-in routines to generate from the 'named' distributions. For instance,

In [ ]:
rng = default_rng(5003)

In [ ]:
yvals = np.zeros((3,300))
yvals[0,:] = rng.normal(size=300)
yvals[1,:] = rng.exponential(size=300)
yvals[2, :] = rng.uniform(size=300)
pdfs = ['Normal', 'Exponential', 'Uniform']
plt.figure(figsize=(12,4), facecolor='0.9')

for i in np.arange(0,3):
    plt.subplot(1, 3, i+1)
    plt.hist(yvals[i,:], density=True, histtype='step')
    plt.title(pdfs[i])
    plt.ylim(0, 1.05)

Notice how the (normalised) histograms look a lot like the corresponding pdfs.

We can even generate values from our own custom distributions. For instance, suppose that we wish to simulate from the following pmf:

$$
P(X=-1) = \frac{1}{2},\quad P(X = 0) = \frac{1}{3}, \quad P(X=1) = \frac{1}{6}
$$

In [ ]:
sim_x = rng.choice(np.array([-1,0,1]), size=1000, replace=True, p=np.array([1/2, 1/3, 1/6]))

In [ ]:
pd.Series(sim_x).value_counts(normalize=True)

If we normalize these counts, the values would resemble the pmf from which they arose. 

The point of simulations is that they can (almost) yield the pmf/pdf of a random variable that we do not know in a 
complicated process. Then we can extract things like means, variances from it. Or we can use it in other ways.

<a id='randomtext'></a>
# Generating Random Text

Here is an example of how we can simulate from distributions to generate random (nonsensical) sentences that are "similar" to actual sentences from a text.

The example uses some ideas from Natural Language Processing which we shall encounter later in the class. First, we install the `nltk` package - the natural language toolkit, and download the books that it uses in examples.

In [ ]:
# !pip install nltk
import nltk

Next, either you run this:

In [ ]:
# nltk.download('genesis')
kjv = nltk.corpus.genesis.words('english-kjv.txt')

Or this:

In [ ]:
# nltk.download('punkt')
from nltk import word_tokenize

with open('../data/english-kjv.txt') as f:
    out = f.read()

kjv = word_tokenize(out)

The object `kjv` contains all the words and punctuations from the King James Bible. Here are the first 20 words:

In [ ]:
' '.join(kjv[:20])

In [ ]:
rng.choice(kjv, size=20)

Next, we extract all bigrams and create a dictionary of frequency distributions. Each frequency distribution tabulates the occurences of the *next* word.

In [ ]:
bg = nltk.bigrams(kjv)
cfd = nltk.ConditionalFreqDist(bg)

Suppose we are interested in what words/tokens come after the word 'And' in the bible:

In [ ]:
cfd['And']

Thus 'he' is the most likely word after 'And'. The second most likely pairing would be 'And the', and so on.

In [ ]:
cfd['And'].keys()

In [ ]:
cfd['And'].values()

We can convert this into a pmf, just by normalising to ensure that it sums to 1. Then we are ready to generate a pairing once we know that the first word is 'And'. With the second word, we turn to its frequency distribution and simulate the third word, and so on.

In [ ]:
ds.generate_model(cfd, 'And', 20)

<a id='des'></a>
# 2. Discrete Event Simulations
<a href=#top>(back to top)</a>

One of the most practical uses of simulation is in Discrete Event Simulations or DES. These are representations of real world. They can be very simple, or they can be ultra complex. They are useful in generating observations in order to study a phenomena, or to perform scenario planning.

Let's study one such latter application and, in the process, learn about a powerful Python Package for DES. It is known as `SimPy`.

## Sandwich Shop

Suppose we operate 'Shirley's Sandwich Shop'. It is a simple kiosk, with one counter serving customers. We receive feedback on social media that the queue is too long during the peak 3 hours from 11am to 2pm. Many customers are being turned away. We would like to investigate ways in which we can solve this issue.

This simplified process has been coded into the IND5003 simulation module. Let's take a look at how it runs first. First, we initialise a dictionary to store the output variables: arrival times, waiting times and service times. Next, we initialise an environment to run the process in. Finally, we create the single "service counter" within the environment. 

In [ ]:
cdict = {}
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)

In [ ]:
env.process(ds.s1(env, 2, counter, 1.0, cdict, True))

In [ ]:
env.run(until=180)  # until=15, 30, ..., 180

In [ ]:
cdict

We can convert the output into a dataframe to study it further.

In [ ]:
cdf = pd.DataFrame.from_dict(cdict, orient='index')
cdf.columns = ['arr', 'wait', 'serv']

In [ ]:
cdf

In [ ]:
plt.figure(figsize=(12,6))
plt.hlines(np.concatenate((np.arange(cdf.shape[0]) + 1, np.arange(cdf.shape[0]) + 1)), 
           np.concatenate((cdf.arr, cdf.arr + cdf.wait)), 
           np.concatenate((cdf.arr + cdf.wait, cdf.arr + cdf.wait + cdf.serv)), colors=['r']*cdf.shape[0] + ['b']*cdf.shape[0]);
plt.title('Customer arrival, service and departure over time');
plt.xlabel('Time in minutes past 11');

In [ ]:
cdf.describe()

In [ ]:
np.sum(cdf.wait.isna())

In [ ]:
cdf.info()
#cdf.dropna().wait.describe()

Let us carry out this simulation 100 times to understand:
1. The median wait-time for customers who were eventually served.
2. The number of customers who did not get served.

In [ ]:
mwt = []
unserved = []

for ii in np.arange(0, 100):
    cdict = {}
    env = simpy.Environment()
    counter = simpy.Resource(env, capacity=1)
    env.process(ds.s1(env, 2, counter, 1.0, cdict, False))
    env.run(until=180)
    
    cdf = pd.DataFrame.from_dict(cdict, orient='index')
    cdf.columns = ['arr', 'wait', 'serv']
    
    unserved.append(np.sum(cdf.wait.isna()))
    mwt.append(cdf.dropna().wait.median())
    

In [ ]:
pd.DataFrame({'mwt':mwt, 'unserved':unserved}).describe()

In [ ]:
pd.DataFrame({'mwt':mwt, 'unserved':unserved}).unserved.plot(kind='hist');

<a id='simpy'></a>
## About SimPy

SimPy is a DES software/framework in Python. In it, the behaviour of active components such as customers is modelled as a process. Processes can interact with one another, and can start process of their own. All processes live in an environment, and they react and trigger events.

Processes are written as generator functions. They contain a `yield` statement that can suspend the execution of a function while waiting for an event to complete.

In [ ]:
def gen_x():
    x = ['Hello', 'who', 'are', 'you']
    for ii in x:
        yield ii  # change yield to return and see what happens

In [ ]:
gg = gen_x()

gg is now a generator object. It will run until the `yield` statement, at which point it returns `ii`, and suspend all variables, every time you call `next` on it.

In [ ]:
next(gg)

In [ ]:
def gen_inf_x():
    ii = 0
    while True:
        ii += 1
        yield ii

In [ ]:
gg2 = gen_inf_x()

In [ ]:
next(gg2)

 Here's a little more detail on [generators](https://www.oreilly.com/content/2-great-benefits-of-python-generators-and-how-they-changed-me-forever/).

In SimPy, processes have to be written as generator function. Here is a simple process &mdash; a car alternating between driving and parking.

In [ ]:
def car(env):
    while True:
        print(f'Start parking at {env.now}')
        yield env.timeout(5)
        
        print(f'Start driving at {env.now}')
        yield env.timeout(2)

In [ ]:
cc = car(env)

In [ ]:
env = simpy.Environment()
env.process(car(env))

So far, we have defined a car process, created the generator object and attached it to the environment. Now we run it for 15 units of time.

In [ ]:
env.run(until=15)

Processes are too simplistic if they do not interact. One way of interacting is to compete for a shared resource. In Shirley's Sandwich Shop, the customers are competing for a counter service.

The first process in that first configuration is the source process (see the function s1). This is the fountain of customers. A customer process (function c1) is created and attached to the environment at random time intervals. The intervals follow an exponential distribution.

Each customer, upon arrival, tries to get service. If there is a queue already there, he/she has to join the queue and wait. Once the customer is at the head of the queue, it takes a random amount of time for him to finish his job. The random amount of time is 1.5 + an Exponential random variable with mean 1. 

<a id='rc'></a>
## Reneging Customers

Now let us add another slight complication. We are going to assume that customers are not as patient as we hoped they would be. If they have been waiting in line too long, they are liable to leave the shop out of frustration. We assume that their "patience" is a uniform random variable between 0.5 and 4.0 minutes. Finally, to gain some indication of how much business is being lost, we represent the amount of money a customer was willing to spend when he walked into the store as a gamma random variable. Finally, we monitor the queue-length every time a customer arrives in as well.

In [ ]:
cdict = {}
env = simpy.Environment()
counter = simpy.Resource(env, capacity=1)

env.process(ds.s2(env, 2, counter, 1.0, cdict, True))
env.run(until=180)  # until=15

Let us construct a data frame to study the output for this run.

In [ ]:
cdf = pd.DataFrame.from_dict(cdict, orient='index')
cdf.columns = ['arr', 'wait', 'serv', 'ren', 'amt_spent', 'qlen']
cdf

In [ ]:
plt.figure(figsize=(12,6))
plt.hlines(np.concatenate((np.arange(cdf.shape[0]) + 1, np.arange(cdf.shape[0]) + 1)), 
           np.concatenate((cdf.arr, cdf.arr + cdf.wait)), 
           np.concatenate((cdf.arr + cdf.wait, cdf.arr + cdf.wait + cdf.serv)), colors=['r']*cdf.shape[0] + ['b']*cdf.shape[0]);
plt.title('Customer arrival, service (or renege) and departure over time');
plt.xlabel('Time in minutes past 11');

In [ ]:
cdf.describe()

In [ ]:
amt = cdf[cdf.ren == 1].amt_spent.sum()
print(f'Num of reneged customers: {int(cdf.ren.sum())}')
print(f'Amt. potentially earned : ${amt:5.2f}')

Here is a plot of how queue length varied during the lunch period.

In [ ]:
plt.figure(figsize=(12,4))
plt.step(np.arange(cdf.shape[0]), cdf.qlen); plt.title('Queue length variation over time');
plt.xlabel('Customer arrival count');

Let us perform a simulation once more, for 100 iterations

In [ ]:
amt = []
reneged = []
served = []

for ii in np.arange(0, 100):
    cdict = {}
    env = simpy.Environment()
    counter = simpy.Resource(env, capacity=1)
    env.process(ds.s2(env, 2, counter, 1.0, cdict, False))
    env.run(until=180)
    
    cdf = pd.DataFrame.from_dict(cdict, orient='index')
    cdf.columns = ['arr', 'wait', 'serv', 'ren', 'amt_spent', 'qlen']
    
    amt_here = cdf[cdf.ren == 1].amt_spent.sum()
    reneged.append(cdf.ren.sum())
    amt.append(amt_here)
    served.append(np.sum(-cdf.serv.isna()))

In [ ]:
pd.DataFrame({'amt':amt, 'reneged':reneged, 'served':served}).describe()

Simulation output is typically used to compute the mean of a process. For instance, we can use the arithmetic mean above to estimate the mean amount of dollars lost, the mean number of customers who reneged, and the mean number of customers served. We can use the sample variance to compute confidence intervals if we wish too. 

Suppose the sandwich shop now wishes to answer two questions:
1. If the shop wishes to serve an average of 70 customers per day, by how much should it extend lunch time?
2. If the shop can reduce its service time by 20%, how many more customers can it serve each day?

If you make changes to functions in a module (e.g., `simulation.py`), you can use the following code to reload the module:

In [ ]:
import importlib
importlib.reload(ds)

## Simulation-Based Modeling

In simulation-based modeling, we have several things to consider. 

1. What input distributions should I use?
2. How complicated does my simulation need to be?
3. How do I know it is giving the right results?
4. How many simulations should I run?

Here are some general guidelines to begin with your simulation model:

1. Start simple.
2. Use your own data to decide what distributions should be used.
3. Try with different distributions to see how sensitive your results are to those choices.
4. Add more and more realistic layers to our simulation as you proceed.
5. Remember that simulation is used in estimating mean values. You will be able to estimate standard deviations around your estimate too.

<a id='ref'></a>
# 3. References
<a href=#top>(back to top)</a>

1. [Simulation](https://www.amazon.com/Simulation-Sheldon-M-Ross/dp/0124158250) by Sheldon Ross is very useful for learning about simulation techniques and concepts. It is available online from our [NUS library](https://linc.nus.edu.sg/record=b3254096).
2. [SimPy](https://simpy.readthedocs.io/en/latest/) documentation. The examples section contains several useful examples that you can customise for your particular case to begin with.
3. [Generators](https://docs.python.org/3/howto/functional.html?highlight=generator#generators) straight from the horse's mouth.
3. More about [generators](https://www.oreilly.com/content/2-great-benefits-of-python-generators-and-how-they-changed-me-forever/).